# Q1 - PCA

## Import the libraries

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib inline

## (A) Load the Data

In [ ]:
def data_and_headers(filename):
    data = None
    with open(filename) as fp:
        data = [x.strip().split(',') for x in fp.readlines()]
    headers = data[0]
    headers = np.asarray(headers)
    class_field = len(headers) - 1
    data_x = [[float(x[i]) for i in range(class_field)] for x in data[1:]]
    data_x = np.asarray(data_x)
    data_y = [[str(x[i]) for i in range(class_field, class_field + 1)] for x in data[1:]]
    data_y = np.asarray(data_y)
    return headers, data_x, data_y

In [ ]:
headers, train_x, train_y = data_and_headers('Data' + os.sep + 'hw2q1_train.csv')
headers, test_x, test_y = data_and_headers('Data' + os.sep + 'hw2q1_test.csv')

In [ ]:
print('Training Data')
print('Number of features - ' + str(train_x.shape[1]))
print('Number of target features - ' + str(train_y.shape[1]))
print('Number of observations - ' + str(train_x.shape[0]))
print('Number of observations in category R - ' + str(train_y[train_y=='R'].shape[0]))
print('Number of observations in category M - ' + str(train_y[train_y=='M'].shape[0]))
print()
print('Testing Data')
print('Number of features - ' + str(test_x.shape[1]))
print('Number of target features - ' + str(test_y.shape[1]))
print('Number of observations - ' + str(test_x.shape[0]))
print('Number of observations in category R - ' + str(test_y[test_y=='R'].shape[0]))
print('Number of observations in category M - ' + str(test_y[test_y=='M'].shape[0]))

## (B) Normalization and PCA

In [ ]:
def normalize(data, minima, maxima):
    normal = np.copy(data)
    normal = (normal - minima) / (maxima - minima)
    return normal

In [ ]:
normal_train = normalize(train_x, np.amin(train_x, axis=0), np.amax(train_x, axis=0))
normal_test = normalize(test_x, np.amin(train_x, axis=0), np.amax(train_x, axis=0))

### (i) Covariance of Training Dataset

In [ ]:
covariance = np.cov(normal_train, rowvar=False)
fig, axes = plt.subplots(nrows=1, ncols=1)
fig.set_figheight(12)
fig.set_figwidth(15)
im = axes.pcolor(covariance, cmap='CMRmap')
fig.colorbar(im, ax=axes)
plt.show()

### (ii) Eigenvalue and Eigenvectors

In [ ]:
print('Size of covariance matrix - ' + str(covariance.shape))
w,v = np.linalg.eig(covariance)
print('Top 5 Eigenvalues - ' + ', '.join(['{:.3f}'.format(x) for x in sorted(w[np.argsort(w)[-5:]])]))

### (iii) Plot of Eigenvalues

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3)
fig.set_figheight(6)
fig.set_figwidth(24)
axes[0].bar(np.arange(60), w)
axes[0].xaxis.grid()
axes[0].yaxis.grid()
axes[0].set_xlabel('eigenvectors')
axes[0].set_ylabel('eigenvalues')
axes[0].set_title('Plot of Variance captured by each eigenvector')
axes[1].bar(np.arange(60), np.cumsum(w))
axes[1].xaxis.grid()
axes[1].yaxis.grid()
axes[1].set_xlabel('eigenvectors')
axes[1].set_ylabel('cumulative sum of eigenvalues')
axes[1].set_title('Cumulative Contribution of eigenvalues')
axes[2].plot(np.arange(60), np.cumsum(w)/np.sum(w))
axes[2].xaxis.grid()
axes[2].yaxis.grid()
axes[2].set_xlabel('eigenvectors')
axes[2].set_ylabel('cumulative sum of eigenvalues (scaled)')
axes[2].set_title('Scaled Cumulative Contribution of eigenvalues')
plt.show()

### (iv) PCA with KNN

In [ ]:
ncomps = [2,4,6,8,10,20,40,60]
pca = [PCA(n_components=x).fit(normal_train) for x in ncomps]
cls = [KNeighborsClassifier(n_neighbors=3, metric='euclidean').fit(pca[i].transform(normal_train), np.ravel(train_y)) for i in range(len(ncomps))]

In [ ]:
pred4 = cls[4].predict(pca[4].transform(normal_test))
true4 = np.ravel(test_y)
with open('q1iv.csv','w') as fp:
    fp.write(', '.join(['Component' + str(i) for i in range(1,ncomps[4]+1)])+', Class, Class\n')
    transformed = pca[4].transform(normal_test)
    for i in range(len(transformed)):
        fp.write(', '.join([str(x) for x in transformed[i]]))
        fp.write(', ' + str(true4[i]))
        fp.write(', ' + str(pred4[i]) + '\n')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1)
fig.set_figheight(6)
fig.set_figwidth(8)
axes.bar(ncomps, [cls[i].score(pca[i].transform(normal_test), np.ravel(test_y)) for i in range(len(ncomps))])
axes.set_xlabel('Number of Components')
axes.set_ylabel('Accuracy of Prediction')
axes.set_title('Performance of Classification for all components')
plt.grid()
plt.show()

## (C) Standardization and PCA

In [ ]:
def standardize(data, mean, sd):
    standard = np.copy(data)
    standard = (standard - mean) / sd
    return standard

In [ ]:
standard_train = standardize(train_x, np.mean(train_x, axis=0), np.std(train_x, axis=0))
standard_test = standardize(test_x, np.mean(train_x, axis=0), np.std(train_x, axis=0))

### (i) Covariance of Training Dataset

In [ ]:
covariance = np.cov(standard_train, rowvar=False)
fig, axes = plt.subplots(nrows=1, ncols=1)
fig.set_figheight(12)
fig.set_figwidth(15)
im = axes.pcolor(covariance, cmap='CMRmap')
fig.colorbar(im, ax=axes)
plt.show()

### (ii) Eigenvalue and Eigenvectors

In [ ]:
print('Size of covariance matrix - ' + str(covariance.shape))
w,v = np.linalg.eig(covariance)
print('Top 5 Eigenvalues - ' + ', '.join(['{:.3f}'.format(x) for x in sorted(w[np.argsort(w)[-5:]])]))

### (iii) Plot of Eigenvalues

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3)
fig.set_figheight(6)
fig.set_figwidth(24)
axes[0].bar(np.arange(60), w)
axes[0].xaxis.grid()
axes[0].yaxis.grid()
axes[0].set_xlabel('eigenvectors')
axes[0].set_ylabel('eigenvalues')
axes[0].set_title('Plot of Variance captured by each eigenvector')
axes[1].bar(np.arange(60), np.cumsum(w))
axes[1].xaxis.grid()
axes[1].yaxis.grid()
axes[1].set_xlabel('eigenvectors')
axes[1].set_ylabel('cumulative sum of eigenvalues')
axes[1].set_title('Cumulative Contribution of eigenvalues')
axes[2].plot(np.arange(60), np.cumsum(w)/np.sum(w))
axes[2].xaxis.grid()
axes[2].yaxis.grid()
axes[2].set_xlabel('eigenvectors')
axes[2].set_ylabel('cumulative sum of eigenvalues (scaled)')
axes[2].set_title('Scaled Cumulative Contribution of eigenvalues')
plt.show()

### (iv) PCA with KNN

In [ ]:
ncomps = [2,4,6,8,10,20,40,60]
pca = [PCA(n_components=x).fit(standard_train) for x in ncomps]
cls = [KNeighborsClassifier(n_neighbors=3, metric='euclidean').fit(pca[i].transform(standard_train), np.ravel(train_y)) for i in range(len(ncomps))]

In [ ]:
pred4 = cls[4].predict(pca[4].transform(standard_test))
true4 = np.ravel(test_y)
with open('q1v.csv','w') as fp:
    fp.write(', '.join(['Component' + str(i) for i in range(1,ncomps[4]+1)])+', Class, Class\n')
    transformed = pca[4].transform(standard_test)
    for i in range(len(transformed)):
        fp.write(', '.join([str(x) for x in transformed[i]]))
        fp.write(', ' + str(true4[i]))
        fp.write(', ' + str(pred4[i]) + '\n')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1)
fig.set_figheight(6)
fig.set_figwidth(8)
axes.bar(ncomps, [cls[i].score(pca[i].transform(standard_test), np.ravel(test_y)) for i in range(len(ncomps))])
axes.set_xlabel('Number of Components')
axes.set_ylabel('Accuracy of Prediction')
axes.set_title('Performance of Classification for all components')
plt.grid()
plt.show()